# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 15 2022 2:50PM,239683,20,ALU0007065,Alumier Labs Inc.,Released
1,Jun 15 2022 2:44PM,239682,10,0085773173,ISDIN Corporation,Released
2,Jun 15 2022 2:44PM,239682,10,0085773177,ISDIN Corporation,Released
3,Jun 15 2022 2:44PM,239682,10,0085772909,ISDIN Corporation,Released
4,Jun 15 2022 2:44PM,239682,10,0085772750,ISDIN Corporation,Released
5,Jun 15 2022 2:44PM,239682,10,0085773091,ISDIN Corporation,Released
6,Jun 15 2022 2:44PM,239682,10,0085773093,ISDIN Corporation,Released
7,Jun 15 2022 2:44PM,239682,10,0085773090,ISDIN Corporation,Released
8,Jun 15 2022 2:44PM,239682,10,0085773100,ISDIN Corporation,Released
9,Jun 15 2022 2:44PM,239682,10,0085773112,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
49,239678,Released,37
50,239679,Released,14
51,239680,Released,1
52,239682,Released,9
53,239683,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239678,NaN,NaN,37.0
239679,NaN,NaN,14.0
239680,NaN,NaN,1.0
239682,NaN,NaN,9.0
239683,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239590,0.0,0.0,1.0
239593,0.0,1.0,0.0
239595,0.0,1.0,0.0
239608,9.0,5.0,1.0
239609,3.0,3.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239590,0,0,1
239593,0,1,0
239595,0,1,0
239608,9,5,1
239609,3,3,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239590,0,0,1
1,239593,0,1,0
2,239595,0,1,0
3,239608,9,5,1
4,239609,3,3,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239590,,,1
1,239593,,1,
2,239595,,1,
3,239608,9,5,1
4,239609,3,3,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 15 2022 2:50PM,239683,20,Alumier Labs Inc.
1,Jun 15 2022 2:44PM,239682,10,ISDIN Corporation
10,Jun 15 2022 2:42PM,239680,10,Yusen – 3D Matrix
11,Jun 15 2022 2:35PM,239678,10,ISDIN Corporation
48,Jun 15 2022 2:25PM,239679,19,"AdvaGen Pharma, Ltd"
62,Jun 15 2022 2:18PM,239677,10,"Methapharm, Inc."
67,Jun 15 2022 2:17PM,239675,10,ISDIN Corporation
68,Jun 15 2022 2:11PM,239674,20,"Exact-Rx, Inc."
69,Jun 15 2022 2:09PM,239672,20,Alumier Labs Inc.
98,Jun 15 2022 2:09PM,239673,19,ACG North America LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 15 2022 2:50PM,239683,20,Alumier Labs Inc.,,,1
1,Jun 15 2022 2:44PM,239682,10,ISDIN Corporation,,,9
2,Jun 15 2022 2:42PM,239680,10,Yusen – 3D Matrix,,,1
3,Jun 15 2022 2:35PM,239678,10,ISDIN Corporation,,,37
4,Jun 15 2022 2:25PM,239679,19,"AdvaGen Pharma, Ltd",,,14
5,Jun 15 2022 2:18PM,239677,10,"Methapharm, Inc.",,,5
6,Jun 15 2022 2:17PM,239675,10,ISDIN Corporation,,,1
7,Jun 15 2022 2:11PM,239674,20,"Exact-Rx, Inc.",,,1
8,Jun 15 2022 2:09PM,239672,20,Alumier Labs Inc.,,,29
9,Jun 15 2022 2:09PM,239673,19,ACG North America LLC,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 15 2022 2:50PM,239683,20,Alumier Labs Inc.,1,,
1,Jun 15 2022 2:44PM,239682,10,ISDIN Corporation,9,,
2,Jun 15 2022 2:42PM,239680,10,Yusen – 3D Matrix,1,,
3,Jun 15 2022 2:35PM,239678,10,ISDIN Corporation,37,,
4,Jun 15 2022 2:25PM,239679,19,"AdvaGen Pharma, Ltd",14,,
5,Jun 15 2022 2:18PM,239677,10,"Methapharm, Inc.",5,,
6,Jun 15 2022 2:17PM,239675,10,ISDIN Corporation,1,,
7,Jun 15 2022 2:11PM,239674,20,"Exact-Rx, Inc.",1,,
8,Jun 15 2022 2:09PM,239672,20,Alumier Labs Inc.,29,,
9,Jun 15 2022 2:09PM,239673,19,ACG North America LLC,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 15 2022 2:50PM,239683,20,Alumier Labs Inc.,1,,
1,Jun 15 2022 2:44PM,239682,10,ISDIN Corporation,9,,
2,Jun 15 2022 2:42PM,239680,10,Yusen – 3D Matrix,1,,
3,Jun 15 2022 2:35PM,239678,10,ISDIN Corporation,37,,
4,Jun 15 2022 2:25PM,239679,19,"AdvaGen Pharma, Ltd",14,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 15 2022 2:50PM,239683,20,Alumier Labs Inc.,1.0,NaN,NaN
1,Jun 15 2022 2:44PM,239682,10,ISDIN Corporation,9.0,NaN,NaN
2,Jun 15 2022 2:42PM,239680,10,Yusen – 3D Matrix,1.0,NaN,NaN
3,Jun 15 2022 2:35PM,239678,10,ISDIN Corporation,37.0,NaN,NaN
4,Jun 15 2022 2:25PM,239679,19,"AdvaGen Pharma, Ltd",14.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 15 2022 2:50PM,239683,20,Alumier Labs Inc.,1.0,0.0,0.0
1,Jun 15 2022 2:44PM,239682,10,ISDIN Corporation,9.0,0.0,0.0
2,Jun 15 2022 2:42PM,239680,10,Yusen – 3D Matrix,1.0,0.0,0.0
3,Jun 15 2022 2:35PM,239678,10,ISDIN Corporation,37.0,0.0,0.0
4,Jun 15 2022 2:25PM,239679,19,"AdvaGen Pharma, Ltd",14.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3834537,207.0,56.0,24.0
15,718938,8.0,0.0,0.0
16,239651,3.0,0.0,0.0
18,479271,2.0,0.0,0.0
19,1917091,25.0,8.0,12.0
20,1677653,69.0,49.0,15.0
21,479188,0.0,2.0,0.0
22,718977,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3834537,207.0,56.0,24.0
1,15,718938,8.0,0.0,0.0
2,16,239651,3.0,0.0,0.0
3,18,479271,2.0,0.0,0.0
4,19,1917091,25.0,8.0,12.0
5,20,1677653,69.0,49.0,15.0
6,21,479188,0.0,2.0,0.0
7,22,718977,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,207.0,56.0,24.0
1,15,8.0,0.0,0.0
2,16,3.0,0.0,0.0
3,18,2.0,0.0,0.0
4,19,25.0,8.0,12.0
5,20,69.0,49.0,15.0
6,21,0.0,2.0,0.0
7,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,207.0
1,15,Released,8.0
2,16,Released,3.0
3,18,Released,2.0
4,19,Released,25.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21,22
Status,,,,,,,,
Completed,24.0,0.0,0.0,0.0,12.0,15.0,0.0,0.0
Executing,56.0,0.0,0.0,0.0,8.0,49.0,2.0,0.0
Released,207.0,8.0,3.0,2.0,25.0,69.0,0.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21,22
0,Completed,24.0,0.0,0.0,0.0,12.0,15.0,0.0,0.0
1,Executing,56.0,0.0,0.0,0.0,8.0,49.0,2.0,0.0
2,Released,207.0,8.0,3.0,2.0,25.0,69.0,0.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21,22
0,Completed,24.0,0.0,0.0,0.0,12.0,15.0,0.0,0.0
1,Executing,56.0,0.0,0.0,0.0,8.0,49.0,2.0,0.0
2,Released,207.0,8.0,3.0,2.0,25.0,69.0,0.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()